**Deploying SQL Server 2019 in Kubernetes**

In [9]:
kubectl apply -f https://raw.githubusercontent.com/nocentino/Presentations/master/DataExposed/SQLServer2019CU4.yaml

secret/mssql unchanged
persistentvolumeclaim/pvc-sql-system unchanged
deployment.apps/mssql-deployment configured
service/mssql-deployment unchanged


**Connect to the Instance in Azure Kubernetes Service**

In [2]:
kubectl get service
$SERVICEIP=kubectl get service mssql-deployment -o jsonpath='{.status.loadBalancer.ingress[].ip}'
$PORT=31433
Write-Output $SERVICEIP,$PORT


sqlcmd -S $SERVICEIP,$PORT -U sa -Q "SELECT @@VERSION" -P 'S0methingS@Str0ng!' 

NAME               TYPE           CLUSTER-IP   EXTERNAL-IP    PORT(S)           AGE
kubernetes         ClusterIP      10.0.0.1     <none>         443/TCP           29m
mssql-deployment   LoadBalancer   10.0.13.25   13.86.57.251   31433:31066/TCP   13m
13.86.57.251
31433
                                                                                                                                                                                                                                                                                                            
------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
Microsoft SQL Server 2019 (RTM-CU4) (KB4548597) - 15.0.4033.1 (X64) 
	Mar 14 2020 16:10:35 
	Copyright (C) 2019 Microsoft Corpo

**Create a database**

In [4]:
sqlcmd -S $SERVICEIP,$PORT -U sa -Q "CREATE DATABASE TestDB1" -P 'S0methingS@Str0ng!' 

In [6]:
sqlcmd -S $SERVICEIP,$PORT -U sa -Q 'SELECT name from sys.databases' -P 'S0methingS@Str0ng!'

name                                                                                                                            
--------------------------------------------------------------------------------------------------------------------------------
master                                                                                                                          
tempdb                                                                                                                          
model                                                                                                                           
msdb                                                                                                                            
TestDB1                                                                                                                         

(5 rows affected)


**Crashing SQL Server** - #Crash SQL Server, do this too many times too quickly it will start to back off...initially 15s


In [8]:
sqlcmd -S $SERVICEIP,$PORT -U sa -Q "SHUTDOWN WITH NOWAIT" -P 'S0methingS@Str0ng!'


Server shut down by NOWAIT request from login sa.


In [9]:
sqlcmd -S $SERVICEIP,$PORT -U sa -Q "SELECT @@VERSION" -P 'S0methingS@Str0ng!' 

                                                                                                                                                                                                                                                                                                            
------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
Microsoft SQL Server 2019 (RTM-CU4) (KB4548597) - 15.0.4033.1 (X64) 
	Mar 14 2020 16:10:35 
	Copyright (C) 2019 Microsoft Corporation
	Developer Edition (64-bit) on Linux (Ubuntu 18.04.4 LTS) <X64>                                                                                                       

(1 rows affected)


Delete SQL Server

In [12]:
$PODNAME=kubectl get pods -l app=mssql -o jsonpath='{ .items[0].metadata.name }'
kubectl delete pod $PODNAME

pod "mssql-deployment-7f5b564898-ltpdj" deleted


In [13]:
sqlcmd -S $SERVICEIP,$PORT -U sa -Q "SELECT @@VERSION" -P 'S0methingS@Str0ng!' 

                                                                                                                                                                                                                                                                                                            
------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
Microsoft SQL Server 2019 (RTM-CU4) (KB4548597) - 15.0.4033.1 (X64) 
	Mar 14 2020 16:10:35 
	Copyright (C) 2019 Microsoft Corporation
	Developer Edition (64-bit) on Linux (Ubuntu 18.04.4 LTS) <X64>                                                                                                       

(1 rows affected)


Updating SQL Server

In [ ]:
kubectl apply -f https://raw.githubusercontent.com/nocentino/Presentations/master/DataExposed/SQLServer2019CU5.yaml